In [271]:
import pandas as pd
from nltk.stem import PorterStemmer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import re
import warnings
warnings.filterwarnings('ignore')

In [272]:
d = pd.read_csv('amazon.csv')
d.head()

product_id                                       product_name  \
0  B07JW9H4J1  Wayona Nylon Braided USB to Lightning Fast Cha...   
1  B098NS6PVG  Ambrane Unbreakable 60W / 3A Fast Charging 1.5...   
2  B096MSW6CT  Sounce Fast Phone Charging Cable & Data Sync U...   
3  B08HDJ86NZ  boAt Deuce USB 300 2 in 1 Type-C & Micro USB S...   
4  B08CF3B7N1  Portronics Konnect L 1.2M Fast Charging 3A 8 P...   

                                            category discounted_price  \
0  Computers&Accessories|Accessories&Peripherals|...             ₹399   
1  Computers&Accessories|Accessories&Peripherals|...             ₹199   
2  Computers&Accessories|Accessories&Peripherals|...             ₹199   
3  Computers&Accessories|Accessories&Peripherals|...             ₹329   
4  Computers&Accessories|Accessories&Peripherals|...             ₹154   

  actual_price discount_percentage  rating rating_count  \
0       ₹1,099                 64%     4.2       24,269   
1         ₹349                 43%     4.0       43,994   
2       ₹1,899                 90%     3.9        7,928   
3         ₹699                 53%     4.2       94,363   
4         ₹399                 61%     4.2       16,905   

                                       about_product  \
0  High Compatibility : Compatible With iPhone 12...   
1  Compatible with all Type C enabled devices, be...   
2  【 Fast Charger& Data Sync】-With built-in safet...   
3  The boAt Deuce USB 300 2 in 1 cable is compati...   
4  [CHARGE & SYNC FUNCTION]- This cable comes wit...   

                                             user_id  \
0  AG3D6O4STAQKAY2UVGEUV46KN35Q,AHMY5CWJMMK5BJRBB...   
1  AECPFYFQVRUWC3KGNLJIOREFP5LQ,AGYYVPDD7YG7FYNBX...   
2  AGU3BBQ2V2DDAMOAKGFAWDDQ6QHA,AESFLDV2PT363T2AQ...   
3  AEWAZDZZJLQUYVOVGBEUKSLXHQ5A,AG5HTSFRRE6NL3M5S...   
4  AE3Q6KSUK5P75D5HFYHCRAOLODSA,AFUGIFH5ZAFXRDSZH...   

                                           user_name  \
0  Manav,Adarsh gupta,Sundeep,S.Sayeed Ahmed,jasp...   
1  ArdKn,Nirbhay kumar,Sagar Viswanathan,Asp,Plac...   
2  Kunal,Himanshu,viswanath,sai niharka,saqib mal...   
3  Omkar dhale,JD,HEMALATHA,Ajwadh a.,amar singh ...   
4  rahuls6099,Swasat Borah,Ajay Wadke,Pranali,RVK...   

                                           review_id  \
0  R3HXWT0LRP0NMF,R2AJM3LFTLZHFO,R6AQJGUP6P86,R1K...   
1  RGIQEG07R9HS2,R1SMWZQ86XIN8U,R2J3Y1WL29GWDE,RY...   
2  R3J3EQQ9TZI5ZJ,R3E7WBGK7ID0KV,RWU79XKQ6I1QF,R2...   
3  R3EEUZKKK9J36I,R3HJVYCLYOY554,REDECAZ7AMPQC,R1...   
4  R1BP4L2HH9TFUP,R16PVJEXKV6QZS,R2UPDB81N66T4P,R...   

                                        review_title  \
0  Satisfied,Charging is really fast,Value for mo...   
1  A Good Braided Cable for Your Type C Device,Go...   
2  Good speed for earlier versions,Good Product,W...   
3  Good product,Good one,Nice,Really nice product...   
4  As good as original,Decent,Good one for second...   

                                      review_content  \
0  Looks durable Charging is fine tooNo complains...   
1  I ordered this cable to connect my phone to An...   
2  Not quite durable and sturdy,https://m.media-a...   
3  Good product,long wire,Charges good,Nice,I bou...   
4  Bought this instead of original apple, does th...   

                                            img_link  \
0  https://m.media-amazon.com/images/W/WEBP_40237...   
1  https://m.media-amazon.com/images/W/WEBP_40237...   
2  https://m.media-amazon.com/images/W/WEBP_40237...   
3  https://m.media-amazon.com/images/I/41V5FtEWPk...   
4  https://m.media-amazon.com/images/W/WEBP_40237...   

                                        product_link  
0  https://www.amazon.in/Wayona-Braided-WN3LG1-Sy...  
1  https://www.amazon.in/Ambrane-Unbreakable-Char...  
2  https://www.amazon.in/Sounce-iPhone-Charging-C...  
3  https://www.amazon.in/Deuce-300-Resistant-Tang...  
4  https://www.amazon.in/Portronics-Konnect-POR-1...

In [273]:
d.shape

(1465, 16)

In [274]:
d.rating.unique()

array([4.2, 4. , 3.9, 4.1, 4.3, 4.4, 4.5, 3.7, 3.3, 3.6, 3.4, 3.8, 3.5,
       4.6, 3.2, 5. , 4.7, 3. , 2.8, 3.1, 4.8, 2.3, 0. , 2. , 2.6, 2.9])

In [275]:
d.to_csv('amazon.csv', index=False)

In [276]:
d.columns

Index(['product_id', 'product_name', 'category', 'discounted_price',
       'actual_price', 'discount_percentage', 'rating', 'rating_count',
       'about_product', 'user_id', 'user_name', 'review_id', 'review_title',
       'review_content', 'img_link', 'product_link'],
      dtype='object')

# Relevent Features
- product_id
- product_name
- category
- about_product

In [277]:
df = d[['product_id', 'product_name', 'category', 'about_product']]
df.head()

product_id                                       product_name  \
0  B07JW9H4J1  Wayona Nylon Braided USB to Lightning Fast Cha...   
1  B098NS6PVG  Ambrane Unbreakable 60W / 3A Fast Charging 1.5...   
2  B096MSW6CT  Sounce Fast Phone Charging Cable & Data Sync U...   
3  B08HDJ86NZ  boAt Deuce USB 300 2 in 1 Type-C & Micro USB S...   
4  B08CF3B7N1  Portronics Konnect L 1.2M Fast Charging 3A 8 P...   

                                            category  \
0  Computers&Accessories|Accessories&Peripherals|...   
1  Computers&Accessories|Accessories&Peripherals|...   
2  Computers&Accessories|Accessories&Peripherals|...   
3  Computers&Accessories|Accessories&Peripherals|...   
4  Computers&Accessories|Accessories&Peripherals|...   

                                       about_product  
0  High Compatibility : Compatible With iPhone 12...  
1  Compatible with all Type C enabled devices, be...  
2  【 Fast Charger& Data Sync】-With built-in safet...  
3  The boAt Deuce USB 300 2 in 1 cable is compati...  
4  [CHARGE & SYNC FUNCTION]- This cable comes wit...

In [278]:
df.isna().sum()

product_id       0
product_name     0
category         0
about_product    0
dtype: int64

# Final Goal : product_id + product_name + tags(category + about_product)

### Clean 'category'

In [279]:
df['category'][0]

'Computers&Accessories|Accessories&Peripherals|Cables&Accessories|Cables|USBCables'

In [280]:
def clean(text):
    x = []
    l = text.split('|')
    for i in l:
        x.extend(i.split('&'))
    return list(set(x))

In [281]:
df['category'] = df['category'].apply(clean)
df['category'][0]

['Computers', 'Cables', 'Accessories', 'Peripherals', 'USBCables']

In [282]:
df['category'] = df['category'].apply(lambda x: ' '.join(x))
df['category']

0       Computers Cables Accessories Peripherals USBCa...
1       Computers Cables Accessories Peripherals USBCa...
2       Computers Cables Accessories Peripherals USBCa...
3       Computers Cables Accessories Peripherals USBCa...
4       Computers Cables Accessories Peripherals USBCa...
                              ...                        
1460    WaterPurifiers Accessories Home HomeAppliances...
1461    Rice SmallKitchenAppliances Home HomeAppliance...
1462    Kitchen AirQuality Heating,Cooling Home HeatCo...
1463    ExhaustFans AirQuality Heating,Cooling Home Ki...
1464    SandwichMakers SmallKitchenAppliances Home Hom...
Name: category, Length: 1465, dtype: object

In [283]:
df.head()

product_id                                       product_name  \
0  B07JW9H4J1  Wayona Nylon Braided USB to Lightning Fast Cha...   
1  B098NS6PVG  Ambrane Unbreakable 60W / 3A Fast Charging 1.5...   
2  B096MSW6CT  Sounce Fast Phone Charging Cable & Data Sync U...   
3  B08HDJ86NZ  boAt Deuce USB 300 2 in 1 Type-C & Micro USB S...   
4  B08CF3B7N1  Portronics Konnect L 1.2M Fast Charging 3A 8 P...   

                                            category  \
0  Computers Cables Accessories Peripherals USBCa...   
1  Computers Cables Accessories Peripherals USBCa...   
2  Computers Cables Accessories Peripherals USBCa...   
3  Computers Cables Accessories Peripherals USBCa...   
4  Computers Cables Accessories Peripherals USBCa...   

                                       about_product  
0  High Compatibility : Compatible With iPhone 12...  
1  Compatible with all Type C enabled devices, be...  
2  【 Fast Charger& Data Sync】-With built-in safet...  
3  The boAt Deuce USB 300 2 in 1 cable is compati...  
4  [CHARGE & SYNC FUNCTION]- This cable comes wit...

In [284]:
df['about_product'][0]

"High Compatibility : Compatible With iPhone 12, 11, X/XsMax/Xr ,iPhone 8/8 Plus,iPhone 7/7 Plus,iPhone 6s/6s Plus,iPhone 6/6 Plus,iPhone 5/5s/5c/se,iPad Pro,iPad Air 1/2,iPad mini 1/2/3,iPod nano7,iPod touch and more apple devices.|Fast Charge&Data Sync : It can charge and sync simultaneously at a rapid speed, Compatible with any charging adaptor, multi-port charging station or power bank.|Durability : Durable nylon braided design with premium aluminum housing and toughened nylon fiber wound tightly around the cord lending it superior durability and adding a bit to its flexibility.|High Security Level : It is designed to fully protect your device from damaging excessive current.Copper core thick+Multilayer shielding, Anti-interference, Protective circuit equipment.|WARRANTY: 12 months warranty and friendly customer services, ensures the long-time enjoyment of your purchase. If you meet any question or problem, please don't hesitate to contact us."

In [285]:
df['tags'] = df['category'] + " " +df['about_product']
df['tags'][0]

"Computers Cables Accessories Peripherals USBCables High Compatibility : Compatible With iPhone 12, 11, X/XsMax/Xr ,iPhone 8/8 Plus,iPhone 7/7 Plus,iPhone 6s/6s Plus,iPhone 6/6 Plus,iPhone 5/5s/5c/se,iPad Pro,iPad Air 1/2,iPad mini 1/2/3,iPod nano7,iPod touch and more apple devices.|Fast Charge&Data Sync : It can charge and sync simultaneously at a rapid speed, Compatible with any charging adaptor, multi-port charging station or power bank.|Durability : Durable nylon braided design with premium aluminum housing and toughened nylon fiber wound tightly around the cord lending it superior durability and adding a bit to its flexibility.|High Security Level : It is designed to fully protect your device from damaging excessive current.Copper core thick+Multilayer shielding, Anti-interference, Protective circuit equipment.|WARRANTY: 12 months warranty and friendly customer services, ensures the long-time enjoyment of your purchase. If you meet any question or problem, please don't hesitate to

### Final Dataframe

In [286]:
df = df[['product_id', 'product_name', 'tags']]
df

product_id                                       product_name  \
0     B07JW9H4J1  Wayona Nylon Braided USB to Lightning Fast Cha...   
1     B098NS6PVG  Ambrane Unbreakable 60W / 3A Fast Charging 1.5...   
2     B096MSW6CT  Sounce Fast Phone Charging Cable & Data Sync U...   
3     B08HDJ86NZ  boAt Deuce USB 300 2 in 1 Type-C & Micro USB S...   
4     B08CF3B7N1  Portronics Konnect L 1.2M Fast Charging 3A 8 P...   
...          ...                                                ...   
1460  B08L7J3T31  Noir Aqua - 5pcs PP Spun Filter + 1 Spanner | ...   
1461  B01M6453MB  Prestige Delight PRWO Electric Rice Cooker (1 ...   
1462  B009P2LIL4  Bajaj Majesty RX10 2000 Watts Heat Convector R...   
1463  B00J5DYCCA  Havells Ventil Air DSP 230mm Exhaust Fan (Pist...   
1464  B01486F4G6  Borosil Jumbo 1000-Watt Grill Sandwich Maker (...   

                                                   tags  
0     Computers Cables Accessories Peripherals USBCa...  
1     Computers Cables Accessories Peripherals USBCa...  
2     Computers Cables Accessories Peripherals USBCa...  
3     Computers Cables Accessories Peripherals USBCa...  
4     Computers Cables Accessories Peripherals USBCa...  
...                                                 ...  
1460  WaterPurifiers Accessories Home HomeAppliances...  
1461  Rice SmallKitchenAppliances Home HomeAppliance...  
1462  Kitchen AirQuality Heating,Cooling Home HeatCo...  
1463  ExhaustFans AirQuality Heating,Cooling Home Ki...  
1464  SandwichMakers SmallKitchenAppliances Home Hom...  

[1465 rows x 3 columns]

In [287]:
# df['tags'] = df['product_name'] + " " + df['tags']

In [288]:
ps = PorterStemmer()
def text_preprocess(text):
    new_text = []
    for i in text.split():
        lower = i.lower()
        # lower = re.sub(r'[^a-zA-Z0-9]', '', lower)
        new_text.append(ps.stem(lower))
    return ' '.join(new_text)

In [289]:
df['tags'] = df['tags'].apply(text_preprocess)
df['tags']

0       comput cabl accessori peripher usbcabl high co...
1       comput cabl accessori peripher usbcabl compat ...
2       comput cabl accessori peripher usbcabl 【 fast ...
3       comput cabl accessori peripher usbcabl the boa...
4       comput cabl accessori peripher usbcabl [charg ...
                              ...                        
1460    waterpurifi accessori home homeappli kitchen w...
1461    rice smallkitchenappli home homeappli kitchen ...
1462    kitchen airqual heating,cool home heatconvecto...
1463    exhaustfan airqual heating,cool home kitchen f...
1464    sandwichmak smallkitchenappli home homeappli k...
Name: tags, Length: 1465, dtype: object

In [290]:
# df.duplicated().sum()

In [291]:
# df.drop_duplicates(keep='first', inplace=True)

In [292]:
df.to_csv("amazon_cleaned.csv", index=False)

### Encode the text data

In [293]:
cv = CountVectorizer(max_features=5000, stop_words='english')
cv

CountVectorizer(max_features=5000, stop_words='english')

In [294]:
cv.fit_transform(df['tags'])

<Compressed Sparse Row sparse matrix of dtype 'int64'
	with 96928 stored elements and shape (1465, 5000)>

In [295]:
vectors = cv.fit_transform(df['tags']).toarray()
vectors

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 1, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]], dtype=int64)

In [296]:
vectors.shape

(1465, 5000)

### Find Similarity

In [297]:
similarity = cosine_similarity(vectors)
similarity

array([[1.        , 0.19701536, 0.29150405, ..., 0.0547438 , 0.08572741,
        0.01005343],
       [0.19701536, 1.        , 0.21809032, ..., 0.        , 0.04368781,
        0.03074014],
       [0.29150405, 0.21809032, 1.        , ..., 0.05269548, 0.08595812,
        0.        ],
       ...,
       [0.0547438 , 0.        , 0.05269548, ..., 1.        , 0.31865767,
        0.04270814],
       [0.08572741, 0.04368781, 0.08595812, ..., 0.31865767, 1.        ,
        0.08359985],
       [0.01005343, 0.03074014, 0.        , ..., 0.04270814, 0.08359985,
        1.        ]])

In [298]:
similarity.shape

(1465, 1465)

In [299]:
sorted(similarity[0], reverse=True)

[0.9999999999999999,
 0.9999999999999999,
 0.9999999999999999,
 0.9660423256566093,
 0.9166450796024617,
 0.9166450796024617,
 0.7960962082206358,
 0.7865115284684452,
 0.7378693532969576,
 0.6543247169611435,
 0.5463519640337732,
 0.5420922401472967,
 0.5152557262433944,
 0.5152557262433944,
 0.4906972855702664,
 0.4754794060540156,
 0.4749089130555018,
 0.4517493057856148,
 0.4411585708139352,
 0.41740484439415804,
 0.4140275002564698,
 0.41346253183283205,
 0.4108824924482613,
 0.40920651226317634,
 0.40493651471679876,
 0.38738338421439583,
 0.381741992249091,
 0.381741992249091,
 0.381741992249091,
 0.381741992249091,
 0.381741992249091,
 0.381741992249091,
 0.3810202180547595,
 0.37387198655421316,
 0.3512784551904357,
 0.34076010936653567,
 0.32431168550227596,
 0.3219028804565233,
 0.3219028804565233,
 0.3219028804565233,
 0.319859031980172,
 0.3128940341859361,
 0.3128940341859361,
 0.3111207137504674,
 0.3111207137504674,
 0.30663941104143694,
 0.30663941104143694,
 0.3008960

In [300]:
list(enumerate(similarity[0]))

[(0, 0.9999999999999999),
 (1, 0.19701536496059194),
 (2, 0.29150405053564693),
 (3, 0.23345031408038378),
 (4, 0.381741992249091),
 (5, 0.21346651211814402),
 (6, 0.2740338729508419),
 (7, 0.11308724526847222),
 (8, 0.09952383094015751),
 (9, 0.24353092034178314),
 (10, 0.22960379059598507),
 (11, 0.1907941753514164),
 (12, 0.16072265341718958),
 (13, 0.2568645196271742),
 (14, 0.381741992249091),
 (15, 0.148239808062881),
 (16, 0.037355786758567155),
 (17, 0.1811819876324102),
 (18, 0.18473566592235327),
 (19, 0.060930841954032575),
 (20, 0.2782087864643736),
 (21, 0.10614598850112184),
 (22, 0.09769056374326629),
 (23, 0.23922328978676283),
 (24, 0.09815721868472535),
 (25, 0.06513780996561759),
 (26, 0.09743717866955018),
 (27, 0.21833484168309816),
 (28, 0.2780639991600243),
 (29, 0.22283706466455738),
 (30, 0.1166361700926264),
 (31, 0.24027475681578908),
 (32, 0.2927060185133856),
 (33, 0.21120476888855716),
 (34, 0.20909292779614744),
 (35, 0.24257558195243636),
 (36, 0.3219028

In [301]:
sorted(list(enumerate(similarity[0])), reverse=True, key=lambda x: x[1])[1:11]

[(369, 0.9999999999999999),
 (614, 0.9999999999999999),
 (220, 0.9660423256566093),
 (42, 0.9166450796024617),
 (89, 0.9166450796024617),
 (80, 0.7960962082206358),
 (166, 0.7865115284684452),
 (106, 0.7378693532969576),
 (208, 0.6543247169611435),
 (261, 0.5463519640337732)]

In [302]:
df.iloc[369]

product_id                                             B07JW9H4J1
product_name    Wayona Nylon Braided USB to Lightning Fast Cha...
tags            comput cabl accessori peripher usbcabl high co...
Name: 369, dtype: object

### Final Function

In [303]:
def recommend(product):
    product_index = df[df['product_name'] == product].index[0]
    distance = similarity[product_index]
    product_list = sorted(list(enumerate(distance)), reverse=True, key=lambda x: x[1])[1:11]

    for i in product_list:
        print(df.iloc[i[0]])

In [304]:
recommend("Wayona Nylon Braided USB to Lightning Fast Charging and Data Sync Cable Compatible for iPhone 13, 12,11, X, 8, 7, 6, 5, iPad Air, Pro, Mini (3 FT Pack of 1, Grey)")

product_id                                             B07JW9H4J1
product_name    Wayona Nylon Braided USB to Lightning Fast Cha...
tags            comput cabl accessori peripher usbcabl high co...
Name: 369, dtype: object
product_id                                             B07JW9H4J1
product_name    Wayona Nylon Braided USB to Lightning Fast Cha...
tags            comput cabl accessori peripher usbcabl high co...
Name: 614, dtype: object
product_id                                             B07JH1CBGW
product_name    Wayona Nylon Braided Usb Syncing And Charging ...
tags            comput cabl accessori peripher usbcabl [high c...
Name: 220, dtype: object
product_id                                             B07JW1Y6XV
product_name    Wayona Nylon Braided 3A Lightning to USB A Syn...
tags            comput cabl accessori peripher usbcabl [high c...
Name: 42, dtype: object
product_id                                             B07JH1C41D
product_name    Wayona Nylon Braided (2 Pac